### **Tipos de cálculos:**

- ~~Rate of Change: P(hoy)/P(hoy – n días)-1~~

- ~~RSI (ver librería Análisis Técnico)~~

- ~~Moving average~~

- ~~Standard Deviation~~

- ~~Beta~~

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import app_functions as af
import plots as pl
import utils

import ta

## Params and data Loading

In [5]:
n_quantiles = 10
prices_fp = 'complete_prices.csv'
shift_period = 1
rets_period = 30

In [6]:
prices_df = pd.read_csv(prices_fp,index_col=0)
prices_df = prices_df.sort_index().fillna(limit=7,method='ffill')

In [7]:
monthly_filter = pd.read_csv(r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\PriceClose_vertical\monthly_constituents_filter.csv',index_col=0)
daily_fiter_mask = pd.read_csv(r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\PriceClose_vertical\daily_filter_mask.csv',index_col=0)

In [5]:
daily_dates = utils.create_dates_list('days',end_date=(2023,1,1),as_str=True)
monthly_dates = utils.create_dates_list('months',end_date=(2023,1,1),as_str=True)

In [8]:
monthly_prices = prices_df.loc[monthly_dates]

NameError: name 'monthly_dates' is not defined

In [7]:
def do_all_monthly(df):
    masked_df = utils.apply_mask(df,monthly_filter)
    ranked_df = af.rank_data(masked_df,10,'alto')
    rents_df = af.get_rents_df(ranked_df,monthly_prices,10,1,1)[0]
    return rents_df

## Rate of change / Momentum
---

### 6 month momentum - Daily prices

In [3]:
def calculate_rate_of_change(df:pd.DataFrame,period:int):
    pct_change_df = df.pct_change(period,limit=1)
    return pct_change_df

In [9]:
pct_change_df = calculate_rate_of_change(prices_df,180)

In [10]:
monthly_masked_pct_change_df = utils.apply_mask(pct_change_df,monthly_filter)

In [11]:
ranked_pct_change_df = af.rank_data(monthly_masked_pct_change_df,n_quantiles,'alto')
rents_pct_change_df,i = af.get_rents_df(ranked_pct_change_df,prices_df,n_quantiles,shift_period,rets_period)

2023-03-30 12:10:59.538 
  command:

    streamlit run c:\Users\hugo.perezdealbeniz\AppData\Local\miniconda3\envs\refinitiv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [11]:
pl.plot_NAV_absoluto(rents_pct_change_df)

In [12]:
pl.plot_sharpe(rents_pct_change_df)

### 6 month momentum - Monthly prices

In [13]:
momentum_6mo = prices_df.pct_change(180,limit=1)

In [14]:
done_momentum_6mo = do_all_monthly(momentum_6mo)

In [15]:
pl.plot_NAV_absoluto(done_momentum_6mo)

In [16]:
pl.plot_sharpe(done_momentum_6mo)

### Year momentum on monthly prices

In [17]:
monthly_pct_change = monthly_prices.pct_change(12,limit=1)

In [18]:
done_monthly_pct_change = do_all_monthly(monthly_pct_change)

In [19]:
pl.plot_NAV_absoluto(done_monthly_pct_change)

In [20]:
pl.plot_sharpe(done_monthly_pct_change)

### PCFperShare test

In [21]:
pcf = pd.read_csv(r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\pivoted_data\PCFperShare.csv',index_col=0).dropna(how='all',axis=0)

In [22]:
pcf_ranked = af.rank_data(pcf,10,'bajo')
pcf_rents,i = af.get_rents_df(pcf_ranked,prices_df,10,1,30)
pl.plot_NAV_absoluto(pcf_rents)

## RSI
***

In [23]:
window = 14

In [24]:
def calculate_rsi(df:pd.DataFrame,window):
    rsi_df = df.apply(ta.momentum.rsi,axis=0,window=window,fillna=False)
    return rsi_df
rsi_df = calculate_rsi(prices_df,window)

In [25]:
done_rsi = do_all_monthly(rsi_df)

In [26]:
pl.plot_NAV_absoluto(done_rsi)

In [27]:
pl.plot_sharpe(done_rsi)

## Media Movil
***

In [28]:
rolling_window = 730

In [29]:
def calculate_simple_ma(df, rolling_window):
    sma_df = df.rolling(rolling_window).mean()
    return sma_df
sma_df = calculate_simple_ma(prices_df,rolling_window)

In [30]:
def calculate_exponential_ma(df:pd.DataFrame, rolling_window):
    ema_df = df.ewm(span=rolling_window).mean()
    return ema_df
ema_df = calculate_exponential_ma(prices_df,rolling_window)

In [31]:
done_sma = do_all_monthly(sma_df)

In [32]:
pl.plot_NAV_absoluto(done_sma)

In [33]:
done_ema = do_all_monthly(ema_df)

In [34]:
pl.plot_NAV_absoluto(done_ema)

In [35]:
pl.plot_sharpe(done_sma)

In [36]:
pl.plot_volatilidad(done_ema)

In [37]:
pl.plot_rentabilidad_media(done_ema)

## Beta
---

In [38]:
def calculate_beta(monthly_prices:pd.DataFrame, monthly_mask:pd.DataFrame, window_size_months:int):

    masked_monthly_prices = utils.apply_mask(monthly_prices,monthly_mask) # should probably pass masked monthly prices directly instead of calculating inside the function
    market = masked_monthly_prices.mean(axis=1)

    window = window_size_months # Rolling window size in months
    
    df = monthly_prices.pct_change(limit=1) # Calculate monthly returns
    market_returns = market.pct_change(limit=1)

    covariance = df.rolling(window).cov(market_returns) # Calculate rolling covariance with market
    variance = market_returns.rolling(window).var() # Calculate rolling variance of market

    beta = covariance.div(variance,axis=0) # Calculate rolling beta
    return beta

In [39]:
beta = calculate_beta(monthly_prices,monthly_filter,12)

In [40]:
done_beta = do_all_monthly(beta)

In [41]:
pl.plot_NAV_absoluto(done_beta)

In [42]:
pl.plot_rentabilidad_media(done_beta)

In [43]:
pl.plot_volatilidad(done_beta)

## Standard deviation (rolling volatility)
---

In [44]:
def calculate_rolling_std(df,window_size):

    std_df = df.rolling(window_size).std()
    return std_df

In [45]:
std_df = calculate_rolling_std(prices_df,30)

In [46]:
done_std = do_all_monthly(std_df)

In [47]:
pl.plot_NAV_absoluto(done_std)

In [48]:
pl.plot_sharpe(done_std)

In [49]:
pl.plot_volatilidad(done_std)